<a href="https://colab.research.google.com/github/bradygm/PID-controller-simulator/blob/main/PID_controller_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RoboLaunch Workshop Series: **Intro to PID Control**

In this notebook you will

If you didn't see the presentation linked to this Colab, you can watch it here: [**How to Control a Robot: Intuition for Feedback Control and PID**]() 

### Quick Google Colab Intro

A notebook is a list of cells. Cells contain either explanatory text or executable code and its output. Click a cell to select it.

Below is a **code cell** that prints "Hello World!". Click in the cell to select it and execute the contents in one of the following ways:

* Click the **Play icon** in the left gutter of the cell;
* Type **Cmd/Ctrl+Enter** to run the cell in place;
* Type **Shift+Enter** to run the cell and move focus to the next cell (adding one if none exists); 

In [ ]:
# This is a comment
print('Hello World!')

Hello World!


Did you see the code cell print "Hello World!"? Great!

# Example 1: Single link robot arm

Our first example is going to be a single link robot arm. This is essentially a rod with one pivot at the end. At the pivot point there is a motor we can control in order to get the arm to rotate in either direction. This rotational force is called **torque**.

The angle of the arm is defined as in the image below:

<img src="https://raw.githubusercontent.com/bradygm/PID-controller-simulator/fc1fc67be86c934b0a07869eaddb00a79ff07488/media/armAngle.png" alt="drawing" width="200"/>






## Imports, helper functions, and classes

We will start by running some code that will set everything up for us. You can run all of these cells by just clicking on the **Play icon** below next to the text that reads "*6 cells hidden*". You can also feel free to open the cells later to look through the code and modify. You can collapse the celss again by clicking on the arrow next to the title of this text box. 

##### Imports

In [66]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import math
from IPython.display import HTML, Javascript
from scipy import interpolate
from ipywidgets import interact, interactive, fixed, interact_manual, Video
import ipywidgets as widgets
import time
from scipy import signal

##### CustomInput Class, interpolation function, and display function

In [42]:
class CustomInput:

    def __init__(self, min_val, max_val):
        self.a = widgets.FloatSlider(description='Input',min=min_val, max=max_val)
        self.values = np.array([[]])

        self.start_time = -1.0
        self.first_time = True

    def f(self, a):
        if self.first_time:
            self.first_time = False
            plt.plot(self.values[:,0], self.values[:,1])
            plt.xlabel("Seconds")
            plt.ylabel("Input")
            return

        if self.start_time == -1.0:
            self.start_time = time.time()

        self.values = np.append(self.values, [[time.time()-self.start_time, a]], axis=0)
        plt.plot(self.values[:,0], self.values[:,1], drawstyle="steps-pre")
        plt.xlabel("Seconds")
        plt.ylabel("Input")

    def record(self):
        self.values = np.empty((0,2), float)
        self.start_time = -1.0
        self.first_time = True

        out = widgets.interactive_output(self.f, {'a': self.a})

        # vid_out = widgets.Output(layout={'border': '1px solid black'})
        # with vid_out:
        #     display(Video.from_file("animation.mp4", play=True))
        # widgets.HBox([widgets.VBox([a]), out, vid_out])
        return widgets.HBox([widgets.VBox([self.a]), out])

In [43]:
def interp_values(values, end_time, dt):
    # values_t = np.array(values)
    f = interpolate.interp1d(values[:,0], values[:,1], kind='previous', fill_value="extrapolate")
    time_list = np.arange(0, end_time, dt)
    return f(time_list)

In [118]:
#############################################
# Display the results
def display_results(goal_ang_interp):
    # arm sim
    fig = plt.figure(figsize=(5, 4))
    ax = fig.add_subplot(autoscale_on=False, xlim=(-my_robot_arm.length*1.2, my_robot_arm.length*1.2), ylim=(-my_robot_arm.length*1.2, my_robot_arm.length*1.2))
    ax.set_aspect('equal')

    line, = ax.plot([], [], 'o-', lw=6)
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)


    def animate_arm(i):
        thisx = [0, my_robot_arm.length*np.sin(ang_hist[i])]
        thisy = [0, my_robot_arm.length*np.cos(ang_hist[i])]

        line.set_data(thisx, thisy)
        time_text.set_text(time_template % (i*my_robot_arm.timestep))
        return line, time_text

    ani_arm_vid = animation.FuncAnimation(
        fig, animate_arm, len(ang_hist), interval=my_robot_arm.timestep*1000, blit=True)
    # gitwriter = animation.ImageMagickWriter()
    # writergif = animation.PillowWriter(fps=50) 
    # ani_arm_vid.save('animation3.gif', writer=writergif)
    plt.close()

    # arm postion vs goal position
    fig = plt.figure(figsize=(6, 4))
    ax = fig.add_subplot(autoscale_on=False, xlim=(0, end_time), ylim=(-np.pi, np.pi))
    plt.title("Arm Angle and Goal Angle")
    plt.xlabel("Seconds")
    plt.ylabel("Angle (Radians)")
    ax.grid()

    goal_list_size = len(goal_ang_interp)

    line2, = ax.plot([], [], '--', lw=2)
    line2.set_label('Arm Angle')
    line3, = ax.plot(time_list[0], ang_hist[0], 'o', lw=2)
    if goal_list_size != 0:
        line4, = ax.plot(time_list[0:goal_list_size-1], goal_ang_interp[0:goal_list_size-1], '--', lw=2)
        line4.set_label('Arm Goal Angle')
    time_template2 = 'time = %.1fs'
    time_text2 = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    # ax.legend()


    def animate_arm2(i):
        line2.set_data(time_list[0:i], ang_hist[0:i])
        line3.set_data(time_list[i], ang_hist[i])
        time_text2.set_text(time_template2 % (i*my_robot_arm.timestep))

        return line2, time_text2

    ani_arm_vid2 = animation.FuncAnimation(
        fig, animate_arm2, len(ang_hist), interval=my_robot_arm.timestep*1000, blit=True)
    plt.close()

    #################################
    # display each animation next to each other
    window1 = widgets.Output(layout={'border': '1px solid black'})
    window2 = widgets.Output(layout={'border': '1px solid black'})
    start = time.time()
    with window1:
        if animate_sim:
            display(HTML(ani_arm_vid.to_html5_video()))
    with window2:
        if animate_plots:
            display(HTML(ani_arm_vid2.to_html5_video()))
        else:
            fig = plt.figure(figsize=(6, 4))
            ax = fig.add_subplot(autoscale_on=False, xlim=(0, end_time), ylim=(-np.pi, np.pi))
            ax.grid()
            plt.title("Arm Angle and Goal Angle")
            plt.xlabel("Seconds")
            plt.ylabel("Angle (Radians)")
            if goal_list_size != 0:
                line2, = ax.plot(time_list[0:list_len-1], goal_ang_interp[0:list_len-1], '--', lw=2)
                line2.set_label('Arm Goal Angle')
            line3, = ax.plot(time_list[0:list_len-1], ang_hist[0:list_len-1], '--', lw=2)
            line3.set_label('Arm Angle')
            ax.legend()
            plt.show()
        
    print(str(time.time()-start) + " sec")

    # FFwriter = animation.FFMpegWriter()
    # ani_arm_vid.save('animation.mp4', writer = FFwriter)
    # gitwriter = animation.ImageMagickWriter()
    # ani_arm_vid.save('animation.gif', writer=gitwriter)
    # ani_arm_vid2.save('animation2.mp4', writer = FFwriter)
    # widgets.HBox([widgets.VBox([a]), out, vid_out])
    return widgets.HBox([window1, window2])
    
    

## Define the Robot Arm

Run the cell below to create your robot arm. You can adjust the parameters of the arm with the toggles on the right, but probably just leave them as they are at first.
<!--
Any cell you see with these adjustable parameters can show or hide its associated code. You can show the code by clicking "Show code" at the bottom of the cell or just by double clicking on the cell itself. In order to hide the code, just double click on parameter box or right click on the cell and select **Form → Show/hide code**.
--> 

In [45]:
# @markdown # Robot Arm Parameters { display-mode: "form" }
# @markdown You can adjust parameters using the sliders and form below.
# @markdown Remember you need to **rerun this
# @markdown cell after you pick new parameters**.

# @markdown ---
class RobotArm:

    def __init__(self, start_angle, timestep):
        # @markdown Mass of the robot arm:
        self.mass = 1 #@param{type:"slider", min:0.1, max:10, step:0.1}
        # @markdown Length of the robot arm:
        self.length =  1#@param{type:"slider", min:0.1, max:10, step:0.1}
        # @markdown Damping from friction
        self.damping = 0.1 #@param
        # @markdown Toggle gravity on or off
        self.gravity_on = False #@param{type:"boolean"}
        
        self.state = np.array([start_angle, 0.0])
        self.gravity = 9.8
        self.timestep = timestep

    def state_derivative(self, state, torque):
        ang_position = state[0]
        ang_velocity = state[1]
        if self.gravity_on:
            accelaration = (3/self.mass/self.length**2)*(torque - self.damping*ang_velocity + self.mass*self.gravity*self.length/2*np.sin(ang_position))
        else:
            accelaration = 3*(torque-ang_velocity*self.damping)/(self.mass*self.length**2)

        return np.array([ang_velocity, accelaration])
        
    def step_forward(self, torque): 
        # Using Runge-Kutta algorithm
        k1 = self.state_derivative(self.state, torque)
        k2 = self.state_derivative(self.state + self.timestep/2*k1, torque)
        k3 = self.state_derivative(self.state + self.timestep/2*k2, torque)
        k4 = self.state_derivative(self.state + self.timestep*k3, torque)
        self.state = self.state + self.timestep/6 * (k1 + 2*k2 + 2*k3 + k4)

    def angle_wrap(self, ang):
        return (ang + np.pi) % (2 * np.pi) - np.pi

    def get_ang_position(self):
        return self.state[0]
    
    def get_ang_velocity(self):
        return self.state[1]

## Let's move the arm around!

Now you get to move the arm! You will do this by varying the torque at the pivot of the arm, just like as if there was a motor there. 


First we will create a sequence of torques. Run the cell below and you should see a slider. Drag the slider around and it will save your input in realtime and display it to the graph on the right. Positive torque will add force in the clockwise direction and negative torque in the counter-clockwise direction.

We will be using the **first 10 seconds** of input in the following steps. 


In [56]:
# Creates the CustomInput object for our input torque sequence
custom_torques = CustomInput(-1,1)
# Starts recording a new torque sequence
custom_torques.record()

### Simulate the result

Now we will simulate the effects of the input you recorded.

Keep the default parameters for now, but feel free to modify later. 

In [110]:
#@markdown How long you want the simulation to run for
end_time = 10.0 #@param{type:"slider", min:5.0, max:30, step:1} 
#@markdown The starting angle of the arm
start_angle = 0.0 #@param{type:"slider", min:-3.14, max:3.14, step:0.1}
dt = 0.05 # the length of the time step
#@markdown Toggle animation of figures on or off. Turning all off will result in quick results.
animate_plots = False #@param {type:"boolean"}
animate_sim = True #@param{type:"boolean"}
time_list = np.arange(0, end_time, dt)
list_len = len(time_list)
ang_hist = []
goal_ang_interp = []

# Initialize the RobotArm and save as my_robot_arm
my_robot_arm = RobotArm(start_angle, dt)

# Takes our custom torque input and interpolates it for all time steps
torques_interp = interp_values(custom_torques.values, end_time, dt)

# Loop over the time steps
for i in range(list_len):
    my_robot_arm.step_forward(torques_interp[i])
    ang_hist.append(my_robot_arm.get_ang_position())

display_results(goal_ang_interp)

9.883253812789917 sec


You can go back and try different torque sequences. Run the torque input cell again to put in new sequence and then run the simulate again to see the results. 

### Follow a goal path

Now we will see if we can get the arm to do something a bit challenging. See if you can get the arm to follow as closely as follows in the example below:

<img src="https://raw.githubusercontent.com/bradygm/PID-controller-simulator/main/media/animation.gif" alt="drawing" width="200"/>
<img src="https://raw.githubusercontent.com/bradygm/PID-controller-simulator/main/media/squareWave2.png" alt="drawing" width="240"/>

Obviously the arm can't teleport from one side to the other, but do your best to get your arm to the goal positions as quickly as possible. Go to the right, then stop the arm. Go to the left, and stop the arm. 

This is pretty tricky, but give it a couple attempts.

In order to more quickly see your results from the simulation, you can choose to **temporarily turn off the animations**. You can always turn it back on whenever to see your final results. Or leave it on all the time if you have great patience 😉.

### PID Controller Magic

That was hard, right? Now that we have tried a couple times to get the arm to follow the goal path, we are now going to write a controller to have it do the hard work for us!

In [30]:
custom_goal_angle = CustomInput(-3.14,3.14)
custom_goal_angle.record()

#### Controller


Probably split up the controller into different ones. Start with p, then pd. Then we do pid? Or later. Maybe...

In [29]:
class PID_Controller:

    def __init__(self, k_p=1, k_i=0, k_d=0):
        self.control_limit = 0 #not currently used
        self.error_sum = 0

        self.k_p = k_p
        self.k_i = k_i
        self.k_d = k_d

    def p_control(self, desired_position, current_position):
        error = desired_position-current_position
        return self.k_p*error

    def pd_control(self, desired_position, current_position, current_velocity):
        error = desired_position-current_position
        return self.k_p*error - self.k_d*current_velocity

    def pid_control(self, desired_position, current_position, current_velocity):
        error = desired_position-current_position
        self.error_sum += error
        return self.k_p*error + self.k_i*self.error_sum - self.k_d*current_velocity

In [40]:
end_time = 10.0 #@param{type:"slider", min:5.0, max:30, step:1}
start_angle = 0.1 #@param{type:"slider", min:-3.14, max:3.14, step:0.1}
k_p = 6.71 #@param{type:"slider", min:0.0, max:20.0, step:0.01}
k_i = 0.0 #@param{type:"slider", min:0.0, max:5.0, step:0.01}
k_d = 1.94 #@param{type:"slider", min:0.0, max:5.0, step:0.01}
dt = 0.05 #@param
animate_plots = False #@param{type:"boolean"}
animate_sim = True #@param{type:"boolean"}
time_list = np.arange(0, end_time, dt)
list_len = len(time_list)

# Initialize the RobotArm and save as my_robot_arm
my_robot_arm = RobotArm(start_angle, dt)
# Initialize our controller
controller = PID_Controller(k_p, k_i, k_d)

torques_interp = interp_values(custom_torques.values, end_time, dt)
goal_ang_interp = interp_values(custom_goal_angle.values, end_time, dt)


ang_hist = []

for i in range(len(torques_interp)):
    my_robot_arm.step_forward(torques_interp[i])
    # my_robot_arm.step_forward(my_robot_arm.p_control(.5))
    # my_robot_arm.step_forward(controller.pd_control(goal_ang_interp[i], my_robot_arm.get_ang_position(), my_robot_arm.get_ang_velocity()))
    # cur_torque = controller.pid_control(0, my_robot_arm.get_ang_position(), my_robot_arm.get_ang_velocity())
    # my_robot_arm.step_forward(cur_torque)
    # my_robot_arm.step_forward(0)
    ang_hist.append(my_robot_arm.get_ang_position())

display_results(goal_ang_interp)

0.21326613426208496 sec


Try turning gravity on or off. Start at angle and see what it does. 

Remember can turn off animation for quicker. Reminder. 

Double click on this cell to view a working answer if you can't get it to work.

<!-- 

Write your comments here 

-->

<hr>

Then they mess around with the slider being the torque. We can then say how rather than a slider with us puting in the torque, we use a PID controller.

Allow for changing the trajectory. Also a slider? How easily change? Swap out or comment out lines?

Now at the end of example 1 things you could try. 
* Change arm parameters and see how it does with the pid parameters you chose. How does it change?
* Change the start config.
* Add gravity. Gravity things to think about and try. Why didn't work? Link to info on how to modify if want. 
* add limit

# Example 2: Line following robot

Could start with or maybe only have a simple straight line. Car going from left to right. They modify the starting position laterally (starting crosstrack error). Maybe move onto a more interesting track or we have a set of tracks. Would be really cool if they could draw their own paths. 

Ideas
* Could have live plot showing cross track error (can be used for explaining the D or I of PID but also just fun to see)
* Have a benchmark set for them to get a score and maybe have leaderboard optional
* Fun parameters we could expose with sliders or text entry or could just be clearly marked in the code
  * Vehicle speed
  * Max turn rate/angle
  * Look ahead distance when calculating cross track error?

# Notes and thoughts

* How allow people new to coding to even participate?
  * Option to reveal the answers or get them somehow and copy and paste
* Make clear what prerequisits are needed
  * Internet Browser
* Could have a quick intro video they watch before the workshop that gives them an overview of google colab or other background info. They then come prepared. Could have a simple notebook they go through
* How make the quality of our presentation really good? 
  * Zoom clear background thing?
  * OBS?
  * Look at other examples?
* How are we going to run the interactive portion, especially with things being hybrid?
  * Will it be open or do they sort of follow along?
  * How check in?
  * Open and then run through together at end?
  * Breakout rooms?
    * How would that work with the recorded video?

# Indu

3rd Graph: Control input torques_interp

On animated plot: Add goal trajectory ghosted

Feel free to clean display_results()

Any speed ups? But probably okay. 


In [117]:
ang_hist = signal.square(.1*2 * np.pi * time_list)
goal_ang_interp = []
display_results(goal_ang_interp)

8.989454984664917 sec
